In [1]:
import matplotlib
import scipy.io
import numpy as np
import random
import plotly.express as px
from matplotlib import pyplot as plt
import sklearn
try:
    import seaborn as sns
    use_seaborn = True
    sns.set()
except:
    use_seaborn = False